[![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-181717?style=for-the-badge&logo=GitHub&link=https://github.com/Mearman/openalex-docs)](https://github.com/Mearman/openalex-docs)[![Open in GitHub](https://img.shields.io/badge/Open%20in-GitHub-181717?style=for-the-badge&logo=github&link=https://github.com/Mearman/openalex-docs/blob/main/how-to-use-the-api/get-lists-of-entities/README.ipynb)](https://github.com/Mearman/openalex-docs/blob/main/how-to-use-the-api/get-lists-of-entities/README.ipynb)[![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-F9AB00?style=for-the-badge&logo=Google%20Colab&link=https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/how-to-use-the-api/get-lists-of-entities/README.ipynb)](https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/how-to-use-the-api/get-lists-of-entities/README.ipynb)

In [ ]:
%pip install --upgrade "git+https://github.com/Mearman/openalex-python-pydantic-v1.git"
%pip install pandasai

In [ ]:
import json
import pandas as pd
import numpy as np
from openalex_api import Configuration, ApiClient, AutocompleteApi, AuthorsApi, ConceptsApi, FundersApi, InstitutionsApi, PublishersApi, SourcesApi, WorksApi

configuration = Configuration(host="https://api.openalex.org")
autocomplete_api = AutocompleteApi(ApiClient(configuration))
authors_api = AuthorsApi(ApiClient(configuration))
concepts_api = ConceptsApi(ApiClient(configuration))
funders_api = FundersApi(ApiClient(configuration))
institutions_api = InstitutionsApi(ApiClient(configuration))
publishers_api = PublishersApi(ApiClient(configuration))
sources_api = SourcesApi(ApiClient(configuration))
works_api = WorksApi(ApiClient(configuration))

from pandasai import SmartDataframe
from pandasai.llm import OpenAI

In [ ]:
# @title  { run: "auto", display-mode: "form" }
openapi_token = "" # @param {type:"string"}

# Get lists of entities

It's easy to get a list of entity objects from from the API:`/<entity_name>.` Here's an example:

* Get a list of _all_ the concepts in OpenAlex:\
  [`https://api.openalex.org/concepts`](https://api.openalex.org/concepts)

In [ ]:
# https://api.openalex.org/concepts
response = concepts_api.get_concepts(
	
)

df = pd.DataFrame(response.results)
display(df)

In [ ]:
numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)

In [ ]:
try:
	llm = OpenAI(api_token = openapi_token)
	sdf = SmartDataframe(numeric_df, config = { "llm": llm })
	sdf.chat("Plot a chart of this data")
except:
	print("Error when creating SmartDataframe")

This query returns a `meta` object with details about the query, a `results` list of [`Concept`](./../../api-entities/concepts/concept-object.ipynb) objects, and an empty [`group_by`](./../get-groups-of-entities.ipynb) list:

```json
meta: {
    count: 65073,
    db_response_time_ms: 81,
    page: 1,
    per_page: 25
    },
results: [
    // long list of Concept entities
 ],
group_by: [] // empty
```

Listing entities is a lot more useful when you add parameters to [page](./paging.ipynb), [filter](./filter-entity-lists.ipynb), [search](./search-entities.ipynb), and [sort](./sort-entity-lists.ipynb) them. Keep reading to learn how to do that.